# RO47019: Intelligent Control Systems Practical Assignment
* Period: 2022-2023, Q3
* Course homepage: https://brightspace.tudelft.nl/d2l/home/500969
* Instructor: Cosimo Della Santina (C.DellaSantina@tudelft.nl)
* Teaching assistant: Ruben Martin Rodriguez (R.MartinRodriguez@student.tudelft.nl)
* (c) TU Delft, 2023

Make sure you fill in any place that says `YOUR CODE HERE` or `YOUR ANSWER HERE`. Remove `raise NotImplementedError()` afterwards. Moreover, if you see an empty cell, please DO NOT delete it, instead run that cell as you would run all other cells. Please fill in your name(s) and other required details below:

In [ ]:
# Please fill in your names, student numbers, netID, and emails below.
STUDENT_1_NAME = ""
STUDENT_1_STUDENT_NUMBER = ""
STUDENT_1_NETID = ""
STUDENT_1_EMAIL = ""

In [ ]:
# Note: this block is a check that you have filled in the above information.
# It will throw an AssertionError until all fields are filled
assert STUDENT_1_NAME != ""
assert STUDENT_1_STUDENT_NUMBER != ""
assert STUDENT_1_NETID != ""
assert STUDENT_1_EMAIL != ""

### General announcements

* Do *not* share your solutions, and do *not* copy solutions from others. By submitting your solutions, you claim that you alone are responsible for this code.

* Do *not* email questions directly, since we want to provide everybody with the same information and avoid repeating the same answers. Instead, please post your questions regarding this assignment in the correct support forum on Brightspace, this way everybody can benefit from the response. If you do have a particular question that you want to ask directly, please use the scheduled Q&A hours to ask the TA.

* There is a strict deadline for each assignment. Students are responsible to ensure that they have uploaded their work in time. So, please double check that your upload succeeded to the Brightspace and avoid any late penalties.

* This [Jupyter notebook](https://jupyter.org/) uses `nbgrader` to help us with automated tests. `nbgrader` will make various cells in this notebook "uneditable" or "unremovable" and gives them a special id in the cell metadata. This way, when we run our checks, the system will check the existence of the cell ids and verify the number of points and which checks must be run. While there are ways that you can edit the metadata and work around the restrictions to delete or modify these special cells, you should not do that since then our nbgrader backend will not be able to parse your notebook and give you points for the assignment. You are free to add additional cells, but if you find a cell that you cannot modify or remove, please know that this is on purpose.

* This notebook will have in various places a line that throws a `NotImplementedError` exception. These are locations where the assignment requires you to adapt the code! These lines are just there as a reminder for youthat you have not yet adapted that particular piece of code, especially when you execute all the cells. Once your solution code replaced these lines, it should accordingly *not* throw any exceptions anymore.

Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

# Linearization

**Author:** Maximilian Stölzle (M.W.Stolzle@tudelft.nl)

This notebook contains function helping to linearize and then discretize then system. Please complete the corresponding assignment in the `task_2d-1_linearization` notebook.

In [ ]:
from functools import partial
from jax.config import config as jax_config

jax_config.update("jax_platform_name", "cpu")  # set default device to 'cpu'
jax_config.update("jax_enable_x64", True)  # double precision
from jax import debug, jacfwd, jit, lax, vmap
import jax.numpy as jnp
from jax.scipy import linalg
from pathlib import Path
from typing import Callable, Dict, Tuple

from jax_double_pendulum.dynamics import (
    continuous_forward_dynamics,
    continuous_inverse_dynamics,
    discrete_forward_dynamics,
    continuous_linear_state_space_representation,
)

In [ ]:
@partial(
    jit,
    static_argnums=0,
    static_argnames=("continuous_forward_dynamics_fn",),
)
def continuous_state_space_dynamics(
    continuous_forward_dynamics_fn: Callable,
    x: jnp.ndarray,
    tau: jnp.ndarray,
    *args_dynamics,
) -> Tuple[jnp.ndarray, jnp.ndarray]:
    """
    Compute the continuous forward dynamics of the system in decoupled form
    Args:
        continuous_forward_dynamics_fn: function to compute the continuous forward dynamics of the system
            Must have the signature th_dd = continuous_forward_dynamics_fn(th, th_d, tau, *args_dynamics)
        x: system state of shape (4, ) consisting of link angles and link angular velocities
        tau: link torques of shape (2, )
        *args_dynamics: additional arguments to pass to continuous_forward_dynamics_fn

    Returns:
        dx_dt: time derivative of system state of shape (4, ).
            Corresponds to link angular velocities and link angular accelerations
        y: system output of shape (2, ). Corresponds to the link angles.

    """
    # YOUR CODE HERE
    raise NotImplementedError()

    return dx_dt, y

In [ ]:
@partial(
    jit,
    static_argnums=0,
    static_argnames=("continuous_forward_dynamics_fn",),
)
def continuous_linear_state_space_representation_autograd(
    continuous_forward_dynamics_fn: Callable,
    th_eq: jnp.ndarray,
    th_d_eq: jnp.ndarray = jnp.zeros((2,)),
    tau_eq: jnp.ndarray = jnp.zeros((2,)),
    *args_dynamics,
) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Linearize the system about the specified state (th, th_d) and input tau
    and return the linearized system in state space representation

    Args:
        continuous_forward_dynamics_fn: function to compute the continuous forward dynamics of the system
            Must have the signature th_dd = continuous_forward_dynamics_fn(th, th_d, tau, *args)
        th_eq: equilibrium link angles of double pendulum of shape (2, )
        th_d_eq: equilibrium link angular velocities of double pendulum of shape (2, )
        tau_eq: equilibrium link torques of double pendulum of shape (2, )
        *args_dynamics: additional arguments to pass to continuous_state_space_dynamics.
            The same additional arguments are then in turn later passed to continuous_forward_dynamics_fn.

    Returns:
        A: state transition matrix of shape (4, 4)
        B: input matrix of shape (4, 2)
        C: output matrix of shape (2, 4)
        D: feed-through matrix of shape (2, 2)
    """
    # Hint: use `jacfwd` on `continuous_state_space_dynamics` to get the gradients of the state transition
    # and outputs with respect to the state and input respectively.

    # YOUR CODE HERE
    raise NotImplementedError()

    return A, B, C, D

In [ ]:
@jit
def cont2discrete_zoh(
    dt: jnp.ndarray,
    A: jnp.ndarray,
    B: jnp.ndarray,
    C: jnp.ndarray,
    D: jnp.ndarray,
) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Discretize continuous-time system using zero-order hold.
    Please refer to the Scipy documentation of the `cont2discrete` function for some inspiration:
        https://github.com/scipy/scipy/blob/v1.9.3/scipy/signal/_lti_conversion.py#L335-L532

    Args:
        dt: time step of the discrete-time system
        A: continuous-time state transition matrix of shape (4, 4)
        B: continuous-time input matrix of shape (4, 2)
        C: continuous-time output matrix of shape (2, 4)
        D: continuous-time feed-through matrix of shape (2, 2)

    Returns:
        Ad: discrete-time state transition matrix of shape (4, 4)
        Bd: discrete-time input matrix of shape (4, 2)
        Cd: discrete-time output matrix of shape (2, 4)
        Dd: discrete-time feed-through matrix of shape (2, 2)

    """
    # YOUR CODE HERE
    raise NotImplementedError()

    return Ad, Bd, Cd, Dd

In [ ]:
@jit
def linearized_discrete_forward_dynamics(
    Ad: jnp.ndarray,
    Bd: jnp.ndarray,
    Cd: jnp.ndarray,
    Dd: jnp.ndarray,
    th_eq: jnp.ndarray,
    th_d_eq: jnp.ndarray,
    tau_eq: jnp.ndarray,
    dt: float,
    th: jnp.ndarray,
    th_d: jnp.ndarray,
    tau: jnp.ndarray,
) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Compute the discrete forward dynamics of the linearized system.
    Should use the linear, discrete-time state-space description to compute the state of the system at the next
    time-step.
    Args:
        Ad: discrete-time state transition matrix of shape (4, 4)
        Bd: discrete-time input matrix of shape (4, 2)
        Cd: discrete-time output matrix of shape (2, 4)
        Dd: discrete-time feed-through matrix of shape (2, 2)
        th_eq: equilibrium link angles of shape (2, )
        th_d_eq: equilibrium link angular velocities of shape (2, )
        tau_eq: equilibrium link torques of shape (2, )
        dt: time step between the current and the next state [s]
        th_curr: current link angles of double pendulum of shape (2, )
        th_d_curr: current link angular velocities of double pendulum of shape (2, )
        tau: link torques of double pendulum of shape (2, )
    Returns:
        th_next: link angles at the next time step of double pendulum of shape (2, )
        th_d_next: link angular velocities at the next time step of double pendulum of shape (2, )
        th_dd: link angular accelerations between current and next time step of double pendulum of shape (2, )
    """
    # Compute the state (th_next, th_d_next) at the next timestep and the corresponding acceleration `th_dd`
    th_next, th_d_next = jnp.zeros_like(th), jnp.zeros_like(th_d)
    th_dd = jnp.zeros_like(th_d)
    # YOUR CODE HERE
    raise NotImplementedError()

    return th_next, th_d_next, th_dd

In [ ]:
# import feedback controller from controllers.ipynb
from ipynb.fs.full.controllers import ctrl_fb_pd


def closed_loop_fb_continuous_forward_dynamics(
    rp: Dict,
    th: jnp.ndarray,
    th_d: jnp.ndarray,
    tau_ext: jnp.ndarray,
    th_des: jnp.ndarray,
    th_d_des: jnp.ndarray,
    kp_fb: jnp.ndarray = jnp.zeros((2,)),
    kd_fb: jnp.ndarray = jnp.zeros((2,)),
) -> jnp.ndarray:
    """
    Adds a feedback control term to the continuous forward dynamics
    Args:
        rp: dictionary of robot parameters used for evaluating the continuous forward dynamics
        th: link angles of shape (2, )
        th_d: link angular velocities of shape (2, )
        tau_ext: external torques of shape (2, ) applied to the system in addition to the feedback torques
        th_des: desired link angles of shape (2, )
        th_d_des: desired link angular velocities of shape (2, )
        kp_fb: proportional gains of the parallel feedback controller of shape (2, 2)
        kd_fb: derivative gains of the parallel feedback controller of shape (2, 2)

    Returns:
        th_dd: link angular accelerations of shape (2, )

    """
    # YOUR CODE HERE
    raise NotImplementedError()

    return th_dd

In [ ]:
# @jit
def linearize_closed_loop_fb_system_about_trajectory(
    rp: Dict,
    traj_ts: Dict[str, jnp.ndarray],
    kp_fb: jnp.ndarray = jnp.zeros((2, 2)),
    kd_fb: jnp.ndarray = jnp.zeros((2, 2)),
) -> Tuple[jnp.ndarray, jnp.ndarray, jnp.ndarray, jnp.ndarray, jnp.ndarray]:
    """
    Linearize the nonlinear double pendulum system about a given trajectory and return the
    linearized system in state space representation.
    Args:
        rp: dictionary of robot parameters used for evaluating the continuous forward dynamics
        traj_ts: dictionary of time series of trajectories
        kp_fb: proportional gains of the parallel feedback controller of shape (2, 2)
        kd_fb: derivative gains of the parallel feedback controller of shape (2, 2)

    Returns:
        tau_eq_ts: time series of equilibrium torques of shape (N, 2)
        Ad_ts: time series of discrete-time state transition matrices of shape (N, 4, 4)
        Bd_ts: time series of discrete-time input matrices of shape (N, 4, 2)
        Cd_ts: time series of discrete-time output matrices of shape (N, 2, 4)
        Dd_ts: time series of discrete-time feed-through matrices of shape (N, 2, 2)

    """
    # number of time-steps
    N = traj_ts["t_ts"].shape[0]

    # compute the equilibrium torque to follow the trajectory using inverse dynamics
    # Hint: you can access `th_ts`, `th_d_ts`, and `th_dd_ts` in the `traj_ts` dictionary.
    tau_eq_ts = jnp.zeros((N, 2))

    # YOUR CODE HERE
    raise NotImplementedError()

    # transform closed_loop_fb_continuous_forward_dynamics function to cohere to interface
    #   th_dd = closed_loop_fb_continuous_forward_dynamics_fn(th, th_d, tau_ext, th_des, th_d_des)
    closed_loop_fb_continuous_forward_dynamics_fn = partial(
        closed_loop_fb_continuous_forward_dynamics,
        rp,
        kp_fb=kp_fb,
        kd_fb=kd_fb,
    )

    # transform continuous_linear_state_space_representation_autograd function to cohere to the interface
    #   A, B, C, D = cl_lsp_autograd_fn(th_eq, th_d_eq, tau_eq, th_des, th_d_des)
    cl_lsp_autograd_fn = partial(
        continuous_linear_state_space_representation_autograd,
        closed_loop_fb_continuous_forward_dynamics_fn,
    )

    # linearize the closed-loop system at each time-step
    A_ts = jnp.zeros((N, 4, 4))
    B_ts = jnp.zeros((N, 4, 2))
    C_ts = jnp.zeros((N, 2, 4))
    D_ts = jnp.zeros((N, 2, 2))

    # YOUR CODE HERE
    raise NotImplementedError()

    # compute the time step
    dt = jnp.mean(traj_ts["t_ts"][1:] - traj_ts["t_ts"][:-1])

    # discretize the state space system using the zero-order hold method
    Ad_ts = jnp.zeros_like(A_ts)
    Bd_ts = jnp.zeros_like(B_ts)
    Cd_ts = jnp.zeros_like(C_ts)
    Dd_ts = jnp.zeros_like(D_ts)

    # YOUR CODE HERE
    raise NotImplementedError()

    return tau_eq_ts, Ad_ts, Bd_ts, Cd_ts, Dd_ts